# Gestion de Usuario y Perfiles

In [1]:
from pymongo import MongoClient
from datetime import datetime
from bson.objectid import ObjectId
import pandas as pd

In [2]:
client = MongoClient("mongodb://localhost:27017")

In [3]:
db = client["bdnr"]
users = db["users"]
print("Conectado a MongoDB correctamente.")

Conectado a MongoDB correctamente.


In [4]:
users.drop()

In [5]:
users.create_index("email", unique=True)

'email_1'

## Funcion para crear datos

In [6]:
def create_user(username, email, profile_pic_url=None, account_status="Free"):
    user = {
        "username": username,
        "email": email,
        "profile_pic_url": profile_pic_url,
        "account_status": account_status,
        "stats": {
            "streak_days": 0,
            "current_league": None,
            "last_active": datetime.utcnow()
        },
        "learning_courses": [],
        "two_factor": {
            "enabled": False,
            "method": None
        },
        "friends_list": [],
        "achievements": [],
        "privacy_settings": {
            "is_profile_public": True,
            "can_receive_messages": True
        }
    }

    try:
        result = users.insert_one(user)
        print(f"Usuario {username} creado correctamente [userId: {result.inserted_id}].")
        return str(result.inserted_id)

    except Exception as e:
        if "duplicate key error" in str(e):
            print(f"Error: El email '{email}' ya está registrado.")
        else:
            print("Error inesperado:", e)
        return None

## Funciones de Lectura

In [7]:
def get_user(user_id):
    return users.find_one({"_id": ObjectId(user_id)})

In [8]:
def get_all_users():
    return users.find()

In [9]:
def users_dataframe():
    data = get_all_users()
    if not data:
        return pd.DataFrame()
    return pd.DataFrame(data)


## Funciones para Actualizar los datos

In [10]:
def update_streak(user_id, streak_days):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {
            "stats.streak_days": streak_days,
            "stats.last_active": datetime.utcnow()
        }}
    )
    print(f"Streak actualizado para userId {user_id}.")


In [11]:
def update_privacy(user_id, **settings):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {f"privacy_settings.{k}": v for k, v in settings.items()}}
    )
    print(f"Privacidad actualizada para userId {user_id}.")


In [12]:
def add_course(user_id, code, level):
    course = {"code": code, "level": level}
    
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"learning_courses": course}}
    )
    print(f"Curso agregado ({code}, nivel {level}) al usuario {user_id}.")

In [13]:
def add_friend(user_id, friend_id):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"friends_list": friend_id}}
    )
    print(f"Amigo {friend_id} agregado a {user_id}.")

In [14]:
def add_achievement(user_id, achievement_code):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"achievements": achievement_code}}
    )
    print(f"Logro '{achievement_code}' agregado a {user_id}.")

## Pruebas de con datos

### Crear usuarios

In [15]:
id1 = create_user("josefina", "josefina@gmail.com")
id2 = create_user("matias", "matias@gmail.com", "fotoPerfil.jpg", "Plus")
id3 = create_user("juan pedro", "juanpedro@gmail.com")

Usuario josefina creado correctamente [userId: 692f2374137f84aeeff0e0f0].
Usuario matias creado correctamente [userId: 692f2374137f84aeeff0e0f1].
Usuario juan pedro creado correctamente [userId: 692f2374137f84aeeff0e0f2].


In [16]:
create_user("josefina", "josefina@gmail.com")

Error: El email 'josefina@gmail.com' ya está registrado.


### Ver datos

In [17]:
users_dataframe()

,_id,username,email,profile_pic_url,account_status,stats,learning_courses,two_factor,friends_list,achievements,privacy_settings
0,692f2374137f84aeeff0e0f0,josefina,josefina@gmail.com,None,Free,"{'streak_days': 0, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[],[],"{'is_profile_public': True, 'can_receive_messa..."
1,692f2374137f84aeeff0e0f1,matias,matias@gmail.com,fotoPerfil.jpg,Plus,"{'streak_days': 0, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[],[],"{'is_profile_public': True, 'can_receive_messa..."
2,692f2374137f84aeeff0e0f2,juan pedro,juanpedro@gmail.com,None,Free,"{'streak_days': 0, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[],[],"{'is_profile_public': True, 'can_receive_messa..."


In [18]:
get_user(id1)

{'_id': ObjectId('692f2374137f84aeeff0e0f0'),
 'username': 'josefina',
 'email': 'josefina@gmail.com',
 'profile_pic_url': None,
 'account_status': 'Free',
 'stats': {'streak_days': 0,
  'current_league': None,
  'last_active': datetime.datetime(2025, 12, 2, 17, 35, 48, 742000)},
 'learning_courses': [],
 'two_factor': {'enabled': False, 'method': None},
 'friends_list': [],
 'achievements': [],
 'privacy_settings': {'is_profile_public': True, 'can_receive_messages': True}}

### Actualizar datos

#### Actualizar racha

In [19]:
update_streak(id1, 2)

Streak actualizado para userId 692f2374137f84aeeff0e0f0.


In [20]:
get_user(id1)['stats']['streak_days']

2

#### Actualizar privacidad

In [21]:
update_privacy(id1, is_profile_public=False, can_receive_messages=False)
get_user(id1)['privacy_settings']

Privacidad actualizada para userId 692f2374137f84aeeff0e0f0.


{'is_profile_public': False, 'can_receive_messages': False}

In [22]:
update_privacy(id2, can_receive_messages=False)
get_user(id2)['privacy_settings']

Privacidad actualizada para userId 692f2374137f84aeeff0e0f1.


{'is_profile_public': True, 'can_receive_messages': False}

#### Actualizar cursos

In [23]:
add_course(id3, "en", 15)
get_user(id3)['learning_courses']

Curso agregado (en, nivel 15) al usuario 692f2374137f84aeeff0e0f2.


[{'code': 'en', 'level': 15}]

In [24]:
add_course(id3, "fr", 2)
get_user(id3)['learning_courses']

Curso agregado (fr, nivel 2) al usuario 692f2374137f84aeeff0e0f2.


[{'code': 'en', 'level': 15}, {'code': 'fr', 'level': 2}]

#### Actualizar amigos

In [25]:
add_friend(id1, id2)

Amigo 692f2374137f84aeeff0e0f1 agregado a 692f2374137f84aeeff0e0f0.


In [26]:
get_user(id1)['friends_list']

['692f2374137f84aeeff0e0f1']

In [27]:
get_user(id2)['friends_list']

[]

In [28]:
add_friend(id2, id1)

Amigo 692f2374137f84aeeff0e0f0 agregado a 692f2374137f84aeeff0e0f1.


In [29]:
get_user(id2)['friends_list']

['692f2374137f84aeeff0e0f0']

#### Actualizar logros

In [30]:
add_achievement(id3, "Language_Master_EN")
get_user(id3)

Logro 'Language_Master_EN' agregado a 692f2374137f84aeeff0e0f2.


{'_id': ObjectId('692f2374137f84aeeff0e0f2'),
 'username': 'juan pedro',
 'email': 'juanpedro@gmail.com',
 'profile_pic_url': None,
 'account_status': 'Free',
 'stats': {'streak_days': 0,
  'current_league': None,
  'last_active': datetime.datetime(2025, 12, 2, 17, 35, 48, 742000)},
 'learning_courses': [{'code': 'en', 'level': 15}, {'code': 'fr', 'level': 2}],
 'two_factor': {'enabled': False, 'method': None},
 'friends_list': [],
 'achievements': ['Language_Master_EN'],
 'privacy_settings': {'is_profile_public': True, 'can_receive_messages': True}}

#### Ver todos los cambios

In [31]:
users_dataframe()

,_id,username,email,profile_pic_url,account_status,stats,learning_courses,two_factor,friends_list,achievements,privacy_settings
0,692f2374137f84aeeff0e0f0,josefina,josefina@gmail.com,None,Free,"{'streak_days': 2, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[692f2374137f84aeeff0e0f1],[],"{'is_profile_public': False, 'can_receive_mess..."
1,692f2374137f84aeeff0e0f1,matias,matias@gmail.com,fotoPerfil.jpg,Plus,"{'streak_days': 0, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[692f2374137f84aeeff0e0f0],[],"{'is_profile_public': True, 'can_receive_messa..."
2,692f2374137f84aeeff0e0f2,juan pedro,juanpedro@gmail.com,None,Free,"{'streak_days': 0, 'current_league': None, 'la...","[{'code': 'en', 'level': 15}, {'code': 'fr', '...","{'enabled': False, 'method': None}",[],[Language_Master_EN],"{'is_profile_public': True, 'can_receive_messa..."


## Agregar nueva Funcionalidad

In [32]:
def set_daily_xp_goal(user_id, xp_goal):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {"daily_xp_goal": xp_goal}}
    )
    print(f"Meta diaria XP = {xp_goal} asignada al usuario {user_id}.")

In [33]:
set_daily_xp_goal(id1, 30)  
get_user(id1)

Meta diaria XP = 30 asignada al usuario 692f2374137f84aeeff0e0f0.


{'_id': ObjectId('692f2374137f84aeeff0e0f0'),
 'username': 'josefina',
 'email': 'josefina@gmail.com',
 'profile_pic_url': None,
 'account_status': 'Free',
 'stats': {'streak_days': 2,
  'current_league': None,
  'last_active': datetime.datetime(2025, 12, 2, 17, 35, 48, 761000)},
 'learning_courses': [],
 'two_factor': {'enabled': False, 'method': None},
 'friends_list': ['692f2374137f84aeeff0e0f1'],
 'achievements': [],
 'privacy_settings': {'is_profile_public': False,
  'can_receive_messages': False},
 'daily_xp_goal': 30}

In [34]:
get_user(id2)

{'_id': ObjectId('692f2374137f84aeeff0e0f1'),
 'username': 'matias',
 'email': 'matias@gmail.com',
 'profile_pic_url': 'fotoPerfil.jpg',
 'account_status': 'Plus',
 'stats': {'streak_days': 0,
  'current_league': None,
  'last_active': datetime.datetime(2025, 12, 2, 17, 35, 48, 742000)},
 'learning_courses': [],
 'two_factor': {'enabled': False, 'method': None},
 'friends_list': ['692f2374137f84aeeff0e0f0'],
 'achievements': [],
 'privacy_settings': {'is_profile_public': True,
  'can_receive_messages': False}}

In [35]:
users_dataframe()

,_id,username,email,profile_pic_url,account_status,stats,learning_courses,two_factor,friends_list,achievements,privacy_settings,daily_xp_goal
0,692f2374137f84aeeff0e0f0,josefina,josefina@gmail.com,None,Free,"{'streak_days': 2, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[692f2374137f84aeeff0e0f1],[],"{'is_profile_public': False, 'can_receive_mess...",30.0
1,692f2374137f84aeeff0e0f1,matias,matias@gmail.com,fotoPerfil.jpg,Plus,"{'streak_days': 0, 'current_league': None, 'la...",[],"{'enabled': False, 'method': None}",[692f2374137f84aeeff0e0f0],[],"{'is_profile_public': True, 'can_receive_messa...",NaN
2,692f2374137f84aeeff0e0f2,juan pedro,juanpedro@gmail.com,None,Free,"{'streak_days': 0, 'current_league': None, 'la...","[{'code': 'en', 'level': 15}, {'code': 'fr', '...","{'enabled': False, 'method': None}",[],[Language_Master_EN],"{'is_profile_public': True, 'can_receive_messa...",NaN
